In [ ]:
##Seb and Lauren created this very rookie code which doesn't run (sorry!). We've tried to make comments throughout for more specific errors
##or parts we were stuck on

In [18]:
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as mcol
import sys
import os
import cartopy.crs as ccrs
import cartopy.feature as cf
import numpy as np
import iris.quickplot as qplt
from iris import load_cube
from iris import coord_systems
import iris.analysis.cartography
import iris.plot as iplt
import iris.coords as icoords
import iris.coord_categorisation

In [19]:
##lauren's code
##this will feed into the main program to return tasmin (or whatever the FIELD is set to) for each SSP, for each model
def individual_model(model):
    scenario = 'ssp585'
    FILENAME = '/badc/cmip6/data/CMIP6/ScenarioMIP/MOHC/' + model + '/' + scenario + '/r1i1p1f3/day/' + FIELD + '/gn/latest/' + FIELD + '_day_' + model + '_' + scenario + '_r1i1p1f3_gn_20150101-20491230.nc'
    
    cube_ssp585 = iris.load_cube(FILENAME, FIELD) ##this throws an error but I can't see why
    
    scenario = 'ssp245'
    FILENAME = '/badc/cmip6/data/CMIP6/ScenarioMIP/MOHC/' + model + '/' + scenario + '/r1i1p1f3/day/' + FIELD + '/gn/latest/' + FIELD + '_day_' + model + '_' + scenario + '_r1i1p1f3_gn_20150101-20491230.nc'
    
    cube_ssp245 = iris.load_cube(FILENAME, FIELD)
    
    scenario = 'ssp126'
    FILENAME = '/badc/cmip6/data/CMIP6/ScenarioMIP/MOHC/' + model + '/' + scenario + '/r1i1p1f3/day/' + FIELD + '/gn/latest/' + FIELD + '_day_' + model + '_' + scenario + '_r1i1p1f3_gn_20150101-20491230.nc'
    
    cube_ssp126 = iris.load_cube(FILENAME, FIELD)
    
    return cube_ssp585, cube_ssp245, cube_ssp126

In [20]:
##Seb's code
##generating etccdi values
##tasmin.min = TNn, tasmin.max = TNx, tasmin.max > 20'C = TR, tasmax.min = TXn, tasmax.max = TXx
def etccdi(tcube):
    
    ### COMBINING BY MONTH/YEAR
    iris.coord_categorisation.add_month(tcube, 'time' , name='month') # add month time coord
    iris.coord_categorisation.add_year(tcube, 'time' , name='year') # add year time coord
    
    ### CREATING ETCCDII VARIABLES ###
    TNxK = tcube.aggregated_by(['month', 'year'], iris.analysis.MAX) #MONTHLY MAXIMUM OF MINS (TASMIN/night)
    TNx = TNxK - 273.15
    
    TNnK = tcube.aggregated_by(['month', 'year'], iris.analysis.MIN) #MONTHLY MINIMUM OF MINS (TASMIN/night)
    TNn = TNnK - 273.15
    
    ##Commenting out TR because it is measured in days rather than degC
    #TR = tcube.aggregated_by(['month', 'year'], iris.analysis.COUNT, function=lambda values: values > 293.15) #MONTHLY count of values above 293.15K (20degC) (TASMIN/night)
    
    return TNx, TNn #, TR

In [21]:
##plotting function - Lauren's attempt
def plot(cube_list):
    for i, cube in enumerate(cube_list): ##I have based this off some old code for a line plot might not work for a map?
        print(cube.data)
        cube_label = np.str(etccdi[i]) 
        plt.figure()
        ax = plt.axes(projection = ccrs.Robinson())
        ax.add_feature(cf.COASTLINE)
        qplt.contourf(tlast_degC, extend='both', cmap='YlOrRd')
        plt.title(model + ' ' + scenario + ' 2015-2049 ' + etccdi)
        plt.points(tlast, c=tlast.data)
        plt.show()
        
##plotting function - Seb's attempt
def plot_test(cube, ax, fig):
    fig = plt.figure()
    ax = plt.axes(projection = ccrs.Robinson())
    ax.add_feature(cf.COASTLINE)
    qplt.contourf(cube, extend='both', cmap='YlOrRd')
    return ax

In [ ]:
##Seb's code
##Creating subplots of all 3 cubes -  not working properly
fig= plt.figure(figsize=(12,8))
fig.suptitle('T') # Title
ax1 = fig.add_subplot(1,3,1,projection = ccrs.Robinson())
plt.axes(projection = ccrs.Robinson())
qplt.contourf(TNnC, extend='both', cmap='YlOrRd')
#plot_test(TNnC,ax1,fig)
#.set_title('') #1st plot title
ax2 = fig.add_subplot(1,3,2,projection = ccrs.Robinson())
plt.axes(projection = ccrs.Robinson())
qplt.contourf(TNxC, extend='both', cmap='YlOrRd')
#plot_test(TNxC,ax2,fig)
#ax2.set_title('')
#ax3=fig.add_subplot(1,3,3, projection = ccrs.Robinson())
#plot_test(TR,ax3,fig)
#ax3.set_title('')
#fig.tight_layout()

In [23]:
##main program (probably not finished)
MODELNAMES = ['HadGEM3-GC31-LL']
FIELD = 'tasmin'

for model in MODELNAMES:
    cube_ssp585, cube_ssp245, cube_ssp126 = individual_model(model) ##this runs the first function and produces each of these cubes
    TNx, TNn = individual_model(model)

/opt/jaspy/lib/python3.7/site-packages/iris/fileformats/cf.py:798: UserWarning: Missing CF-netCDF measure variable 'areacella', referenced by netCDF variable 'tasmin'
  warnings.warn(message % (variable_name, nc_var_name))


ConstraintMismatchError: no cubes found